# This is test jupyter notebook file, it tests the "tools/common_funcs.py".

In [1]:
!earthengine authenticate

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=l46WclOtFDjuvpNK-QIAPDS7guRFmrDv7is2urCxWmM&tc=lk9ZYeTDo4qMVZEgrXwtebC8FC6MR5zrU0l6wuP3efM&cc=1uR-91nCTWkM6yThi30FbLF9LcRUY9X7dV5JpTYien8

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BWaAjLdpu--G06cicP-zdFJX8xiwQ7Kn933MH8vhkO_Q1LoBZ3rtnQ

Successfully saved authorization token.


In [2]:
# Prepare a bit of packages:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
tool_folder_dir = "/content/gdrive/MyDrive/irp_project_111/source code"
sys.path.append(tool_folder_dir)
!pip install ipytest

Mounted at /content/gdrive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.1 MB/s eta 0:00:00


In [ ]:
"""
Author: Chaofan Wu
Student ID: 02285924
Email: cw522@ic.ac.uk
Project Name: Predicting flood risk in Ghana
Supervisors:
    Sesinam Dagadu(MEng)
    Yves Plancherel(PhD)
Company: SnooCODE
Date: 08/2023
"""
import ipytest
ipytest.autoconfig()
from tools import common_fucs
import pandas as pd
import ee

def test_check_duplicate_rows():
    """Test the check_duplicate_rows function."""
    # Create a data frame for testing
    df1 = pd.DataFrame({'A': [1, 2, 3, 2], 'B': [5, 6, 7, 6]})
    # Case1: Tests whether the function recognises repetition of an entire line
    assert common_fucs.check_duplicate_rows(df1) == True, "Failed to recognise duplicate rows."
    # Case2: Tests if the function recognises duplicates in a particular column
    assert common_fucs.check_duplicate_rows(df1, columns=['A']) == True, "Failed to recognise a duplicate in column 'A'."

    # Case3: Tests if the function will not mark non-duplicate rows/columns as duplicates
    df2 = pd.DataFrame({'A': [1, 2, 3, 4], 'B': [5, 6, 7, 8]})
    assert common_fucs.check_duplicate_rows(df2) == False, "Incorrectly identified non-duplicate rows as duplicates."


def test_discard_duplicates():
    """Tests the discard_duplicates function."""
    # Create a data frame for testing
    df1 = pd.DataFrame({'A': [1, 2, 3, 2], 'B': [5, 6, 7, 6]})

    # Case1: Check if duplicates for the entire row are discarded
    new_df1 = common_fucs.discard_duplicates(df1)
    assert not new_df1.duplicated().any(), "Duplicate rows were not discarded."
    assert new_df1.shape[0] == 3, "Unexpected number of rows after discarding duplicates."

    # Case2: Check if duplicates for a specific column are discarded
    new_df2 = common_fucs.discard_duplicates(df1, columns=['A'])
    assert not new_df2.duplicated(subset=['A']).any(), "Duplicates in column 'A' were not discarded."
    assert new_df2.shape[0] == 3, "Unexpected number of rows after discarding duplicates in column 'A'."

    # Case3: Check with a DataFrame that has no duplicates
    df2 = pd.DataFrame({'A': [1, 2, 3, 4], 'B': [5, 6, 7, 8]})
    new_df3 = common_fucs.discard_duplicates(df2)
    assert not new_df3.duplicated().any(), "No duplicates should be found in df2."
    assert new_df3.shape[0] == 4, "Unexpected number of rows after checking df2 for duplicates."

    # Case4: Check if the function returns a new copy of the DataFrame and does not modify the original in-place
    df3 = pd.DataFrame({'A': [1, 2, 3, 2], 'B': [5, 6, 7, 6]})
    new_df4 = common_fucs.discard_duplicates(df3)
    assert df3.shape[0] == 4, "Original DataFrame should not be modified in-place."


def test_compare_float_lists():
    """Tests the compare_float_lists function."""
    # Case1: Check if function correctly identifies approximately equal lists
    list_a = [0.1 + 0.2, 0.3]
    list_b = [0.3, 0.3]
    assert common_fucs.compare_float_lists(list_a, list_b) == True, "Failed to recognize approximately equal lists."

    # Case2: Check if function identifies lists that are not approximately equal
    list_c = [0.1 + 0.2, 0.3]
    list_d = [0.3, 0.31]
    assert common_fucs.compare_float_lists(list_c,
                                           list_d) == False, "Incorrectly recognized unequal lists as approximately equal."

    # Case3: Check with lists of different lengths
    list_e = [0.1, 0.2, 0.3]
    list_f = [0.1, 0.2]
    assert common_fucs.compare_float_lists(list_e,
                                           list_f) == False, "Lists of different lengths should be identified as not equal."

    # Case4: Check with tolerance parameter
    list_g = [0.1, 0.2, 0.3]
    list_h = [0.1, 0.2, 0.3 + 1e-6]
    assert common_fucs.compare_float_lists(list_g, list_h,
                                           tolerance=1e-5) == True, "Failed to recognize approximately equal lists with tolerance parameter."
    assert common_fucs.compare_float_lists(list_g, list_h,
                                           tolerance=1e-7) == False, "Incorrectly recognized unequal lists as approximately equal with tolerance parameter."


def test_compare_float_2d_lists():
    """Tests the compare_float_2d_lists function."""

    # Case1: Check if function correctly identifies approximately equal 2D lists
    list_a_2d = [[0.1 + 0.2, 0.3], [0.4, 0.5]]
    list_b_2d = [[0.3, 0.3], [0.4, 0.5]]
    assert common_fucs.compare_float_2d_lists(list_a_2d,
                                              list_b_2d) == True, "Failed to recognize approximately equal 2D lists."

    # Case2: Check if function identifies 2D lists that are not approximately equal
    list_c_2d = [[0.1 + 0.2, 0.3], [0.4, 0.5]]
    list_d_2d = [[0.3, 0.3], [0.4, 0.51]]
    assert common_fucs.compare_float_2d_lists(list_c_2d,
                                              list_d_2d) == False, "Incorrectly recognized unequal 2D lists as approximately equal."

    # Case3: Check with 2D lists of different outer lengths
    list_e_2d = [[0.1, 0.2, 0.3], [0.4, 0.5]]
    list_f_2d = [[0.1, 0.2, 0.3]]
    assert common_fucs.compare_float_2d_lists(list_e_2d,
                                              list_f_2d) == False, "2D Lists of different outer lengths should be identified as not equal."

    # Case4: Check with 2D lists of different inner lengths
    list_g_2d = [[0.1, 0.2, 0.3], [0.4, 0.5]]
    list_h_2d = [[0.1, 0.2], [0.4, 0.5]]
    assert common_fucs.compare_float_2d_lists(list_g_2d,
                                              list_h_2d) == False, "2D Lists of different inner lengths should be identified as not equal."

    # Case5: Check with tolerance parameter
    list_i_2d = [[0.1, 0.2, 0.3], [0.4, 0.5 + 1e-6]]
    list_j_2d = [[0.1, 0.2, 0.3], [0.4, 0.5]]
    assert common_fucs.compare_float_2d_lists(list_i_2d, list_j_2d,
                                              tolerance=1e-5) == True, "Failed to recognize approximately equal 2D lists with tolerance parameter."
    assert common_fucs.compare_float_2d_lists(list_i_2d, list_j_2d,
                                              tolerance=1e-7) == False, "Incorrectly recognized unequal 2D lists as approximately equal with tolerance parameter."


def test_convert_to_ee_rectangle():
    """Tests the convert_to_ee_rectangle function."""
    # Initialize the Earth Engine library
    ee.Initialize()

    # Case1: Test with a simple set of coordinates
    coords1 = [[[-0.642576, 8.172062], [-0.502478, 8.172062], [-0.502478, 8.339226], [-0.642576, 8.339226], [-0.642576, 8.172062]]]
    rectangle1 = common_fucs.convert_to_ee_rectangle(coords1)
    assert rectangle1.getInfo()["coordinates"] == [[[-0.642576, 8.172062], [-0.502478, 8.172062], [-0.502478, 8.339226], [-0.642576, 8.339226], [-0.642576, 8.172062]]], "Failed to construct correct rectangle for coords1."



# Start Test:
ipytest.run()

.....                                                                                        [100%]
5 passed in 1.81s


<ExitCode.OK: 0>